# Exercise 14 - Data-Driven Identification using Physics-Informed Neural Networks for a Static Bar
### Task
Modify the file from exercise 14 to solve an inverse problem with a physics-informed neural network. File is adapated from Exercise 13. Changes are marked in <font color='red'>**red**</font>. 
1. implement the `getLossTerms` function for the inverse problem.
2. train the physics-informed neural network to learn the inversion
3. adapt the hyperparameters as needed
4. reproduce the example from Section 4.3 by changing the problem parameters

### Learning goals
- Understand the difference between forward and inverse problems
- Understand how to use physics-informed neural networks for inverse problems
- Gain an intuition about the performance of physics-informed neural networks and how the hyperparameters affect the convergence 


**import libraries & set seed**

In [ ]:
import numpy as np
import torch
from torch.autograd import grad
import time
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(2)

## Utilities

**gradient computation with automatic differentiation**

In [ ]:
def getDerivative(y, x, n):
    """Compute the nth order derivative of y = f(x) with respect to x."""

    if n == 0:
        return y
    else:
        dy_dx = grad(
            y, x, torch.ones(x.size()[0], 1), create_graph=True, retain_graph=True
        )[0]
        return getDerivative(dy_dx, x, n - 1)

**neural network**

In [ ]:
class NN(torch.nn.Module):
    def __init__(
        self,
        inputDimension,
        hiddenDimensions,
        outputDimension,
        activationFunction=torch.nn.Tanh(),
    ):
        super().__init__()

        modules = []

        modules.append(torch.nn.Linear(inputDimension, hiddenDimensions[0]))
        modules.append(activationFunction)
        for i in range(len(hiddenDimensions) - 1):
            modules.append(
                torch.nn.Linear(hiddenDimensions[i], hiddenDimensions[i + 1])
            )
            modules.append(activationFunction)
        modules.append(torch.nn.Linear(hiddenDimensions[-1], outputDimension))

        self.model = torch.nn.Sequential(*modules)

    def forward(self, x):
        return self.model(x)

**initialization of neural network weights**

In [ ]:
def initWeights(m):
    """Initialize weights of neural network with xavier initialization."""
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_uniform_(
            m.weight, gain=torch.nn.init.calculate_gain("tanh")
        )  # adapt if using a different initialization
        m.bias.data.fill_(0.0)

## PINN helper functions

<font color='red'>**stiffness computation**</font>
$$\hat{EA}=F_{NN}(x)$$

In [ ]:
def getStiffness(model, x):
    return model(x)

<font color='red'>**loss term computation**</font>

<font color='red'>the differential equation loss</font>
$$\mathcal{L}_R=\sum_{i=1}^N\bigl(\frac{d}{dx}\hat{EA}\bigl(\frac{du}{dx}\bigr)+p\bigr)^2$$
<font color='red'>boundary condition is already fulfilled by measurement of $u$</font> 

In [ ]:
def getLossTerms(x, u, EA, distLoad):
    raise NotImplementedError()  # your code goes here
    # return differentialEquationLoss

<font color='red'>**cost function computation**</font>
$$C=\mathcal{L}_R$$

In [ ]:
def getCostFunction(lossTerms):
    return lossTerms

## Problem setup

<font color='red'>**physical parameters**</font>

In [ ]:
# your code goes here

**hyperparameters**

currently Adam is selected as optimizer. By commenting the Adam block and uncommenting the LBFGS block, you can enable LBFGS as optimizer.

In [ ]:
Nx = 100  # number of collocation points
hiddenDimensions = [100]  # definition of hidden layers
activationFunction = (
    torch.nn.Tanh()
)  # if this is changed, also adapt the initialization

epochs = 5000  # number of epochs
lr = 5e-3  # learning rate
selectOptimizer = "Adam"

# epochs = 500
# selectOptimizer = "LBFGS"
# lr = 1e-2

**neural network & optimizer setup**

In [ ]:
model = NN(1, hiddenDimensions, 1, activationFunction)
model.apply(initWeights)
if selectOptimizer == "Adam":
    optimizer = torch.optim.Adam(model.parameters(), lr)
elif selectOptimizer == "LBFGS":
    optimizer = torch.optim.LBFGS(model.parameters(), lr)

<font color='red'>**training grid**</red>

In [ ]:
x = torch.linspace(0, L, Nx, requires_grad=True).unsqueeze(1)

<font color='red'>**measurements**</font>

In [ ]:
uMeasured = uAnalytic(x)  # note that u is differentiable due to sampling

## <font color='red'>Training</font>

In [ ]:
costHistory = np.zeros(epochs)

start = time.perf_counter()
start0 = start
for epoch in range(epochs):
    # predict displacements
    EAPred = getStiffness(model, x)

    lossTerms = getLossTerms(x, uMeasured, EAPred, distLoad(x))
    costHistory[epoch] = getCostFunction(lossTerms).detach()

    def closure():
        optimizer.zero_grad()
        EAPred = getStiffness(model, x)
        lossTerms = getLossTerms(x, uMeasured, EAPred, distLoad(x))
        cost = getCostFunction(lossTerms)
        cost.backward(retain_graph=True)
        return cost

    optimizer.step(closure)

    if epoch % 100 == 0:
        elapsedTime = (time.perf_counter() - start) / 100
        string = "Epoch: {}/{}\t\tCost = {:2f}\t\tElapsed time = {:2f}"
        # Format string and print
        print(string.format(epoch, epochs - 1, costHistory[epoch], elapsedTime))
        start = time.perf_counter()
elapsedTime = time.perf_counter() - start0
string = "Total elapsed time: {:2e}\nAverage elapsed time per epoch: {:2f}"
print(string.format(elapsedTime, elapsedTime / epochs))

## Post-processing

<font color='red'>**training history**</font>

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Epochs")
ax.set_ylabel("Cost function $C$")
ax.set_yscale("log")

ax.plot(costHistory, "k", linewidth=2, label="Cost $C$")

ax.grid()
ax.legend()
fig.tight_layout()
plt.show()

<font color='red'>**displacement prediction**</font>

In [ ]:
xTest = torch.linspace(0, L, 1000).unsqueeze(1)
EAPredTest = getStiffness(model, xTest).detach()
EAPred = getStiffness(model, x).detach()

fig, ax = plt.subplots()
ax.set_xlabel("$x$")
ax.set_ylabel("Displacement $u$")

ax.plot(xTest, EAAnalytic(xTest), "gray", linewidth=2, label="Analytical solution")
ax.plot(xTest, EAPredTest, "k:", linewidth=2, label="Prediction")
ax.plot(x.detach(), EAPred, "rs", markersize=6, label="Collocation points")

ax.grid()
ax.legend()
fig.tight_layout()
plt.show()